## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Mataura,NZ,2022-02-05 00:51:09,-46.1927,168.8643,55.47,40,20,3.20,few clouds
1,1,Airai,TL,2022-02-05 00:51:10,-8.9266,125.4092,70.25,70,100,1.63,overcast clouds
2,2,Atasu,KZ,2022-02-05 00:51:10,48.6814,71.6433,6.51,79,5,9.86,clear sky
3,3,Nyuzen,JP,2022-02-05 00:51:11,36.9333,137.5000,34.50,86,100,15.37,snow
4,4,Rikitea,PF,2022-02-05 00:50:16,-23.1203,-134.9692,79.03,67,5,5.95,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current_Description
0,0,Mataura,NZ,2022-02-05 00:51:09,-46.1927,168.8643,55.47,40,20,3.20,few clouds
1,1,Airai,TL,2022-02-05 00:51:10,-8.9266,125.4092,70.25,70,100,1.63,overcast clouds
6,6,Kaitangata,NZ,2022-02-05 00:51:12,-46.2817,169.8464,59.04,49,28,7.29,scattered clouds
7,7,East London,ZA,2022-02-05 00:51:12,-33.0153,27.9116,68.86,85,100,7.43,light rain
8,8,Cape Town,ZA,2022-02-05 00:51:12,-33.9258,18.4232,64.89,68,0,5.01,clear sky
9,9,Najran,SA,2022-02-05 00:51:13,17.4924,44.1277,66.85,71,4,8.57,clear sky
10,10,Pisco,PE,2022-02-05 00:47:28,-13.7000,-76.2167,68.05,83,0,18.41,clear sky
15,15,Dawei,MM,2022-02-05 00:51:15,14.0833,98.2000,74.21,83,82,3.56,broken clouds
23,23,Cidreira,BR,2022-02-05 00:46:19,-30.1811,-50.2056,73.00,91,24,4.97,few clouds
34,34,Lanquin,GT,2022-02-05 00:51:22,15.5667,-89.9667,74.93,98,10,3.29,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                208
City                   208
Country                204
Date                   208
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current_Description    208
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                204
City                   204
Country                204
Date                   204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current_Description    204
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current_Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current_Description,Lat,Lng,Hotel Name
0,Mataura,NZ,55.47,few clouds,-46.1927,168.8643,
1,Airai,TL,70.25,overcast clouds,-8.9266,125.4092,
6,Kaitangata,NZ,59.04,scattered clouds,-46.2817,169.8464,
7,East London,ZA,68.86,light rain,-33.0153,27.9116,
8,Cape Town,ZA,64.89,clear sky,-33.9258,18.4232,
9,Najran,SA,66.85,clear sky,17.4924,44.1277,
10,Pisco,PE,68.05,clear sky,-13.7000,-76.2167,
15,Dawei,MM,74.21,broken clouds,14.0833,98.2000,
23,Cidreira,BR,73.00,few clouds,-30.1811,-50.2056,
34,Lanquin,GT,74.93,clear sky,15.5667,-89.9667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()
hotel_df.count()

City                   204
Country                204
Max Temp               204
Current_Description    204
Lat                    204
Lng                    204
Hotel Name             204
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df=hotel_df.copy()
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current_Description}</dd> and <dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))